# Bibliotecas Necessárias

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from os.path import join

# Importando os Dados Usados

Para este caso, vamos usados o dataset de treino [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic/data?select=train.csv)

In [ ]:
df = pd.read_csv(join("Data","Titanic","train.csv"))

# Separando os dados entre Características e Resultado (features, targets)

In [ ]:
features, target = df.drop(columns=["Survived"]), 